In [1]:
from openai import OpenAI

from dotenv import load_dotenv
import numpy as np
import pandas as pd
import requests
import joblib
import os
import re

import utils

load_dotenv()

True

### Load data

In [2]:
import json

In [3]:
with open(f'data/machine_learning.json', 'r') as f:
    projects = json.load(f)


In [4]:
df = utils.load_bids_df(dataset_name='machine_learning')
df_prepared = utils.prepare_df(df)

### Load estimator

In [5]:
estimator = joblib.load('estimators/ml_bids_estimator.json')

In [6]:
def predict_bids_probs(bids):
    return estimator.predict_proba(bids)[:, 1]

In [13]:
def predict_generated_bid_prob(project_bids, bid_text):
    bid = project_bids.reset_index(drop=True).iloc[0].copy()
    
    bid['bid_sealed'] = False
    bid['bid_highlighted'] = False
    
    # 3 minute after first bid
    bid['bid_time_submitted'] = bid['project_time_submitted'] + 60*10
    
    reputation = user_data['entire_history']['category_ratings']
    
    bid['bid_quality'] = reputation['quality']
    bid['bid_communication'] = reputation['communication']
    bid['bid_professionalism'] = reputation['professionalism']
    
    bid['bid_score'] = np.max(project_bids['bid_score'])
    
    # Transforming generated bid text in features
    text_before, text_after = bid_text.split('Description:')
    
    period = re.findall('(?<=Period of work: )\d+?', text_before)[0]
    bid['bid_period'] = int(period)
    
    amount = re.findall('(?<=Price of work: \$)\d*(?=;)', text_before)[0]
    bid['bid_amount'] = float(amount)
    
    bid['bid_description'] = text_after
    
    one_bid_df = pd.DataFrame(columns=df.columns, data=[bid.values.tolist()])

    one_bid_df = utils.prepare_df(one_bid_df)
    pred = predict_bids_probs(one_bid_df)[0]
    
    return pred, bid

### Get user data

In [8]:
access_token = '' or os.getenv('FREELANCER_ACCESS_TOKEN')
# /users/0.1/
user_id = '70505411'
url = f"https://www.freelancer.com/api/users/0.1/reputations/"

headers = {'freelancer-oauth-v1': access_token}
params = {"users[]": [int(user_id)]}
response = requests.request("GET", url, headers=headers, params=params)

user_data = response.json()['result'][user_id]

### ChatGPT API generation

In [9]:
API_KEY = '' or os.getenv('OPENAI_API_KEY')

In [10]:
client = OpenAI(api_key=API_KEY)

system_content = ("You are a freelancer which specializes in machine learning."
                  "You have an understanding of the field, because you have learnt a lot, "
                  "but you haven't done anything important in this field."
                  "You work on freelancer.com and you want to be adwarded for this project."
                  "Propose to the client the way of solving the problem."
                  "You can write no more than 100 words."
                  "You need to answer by such model: Period of work: [number] days;"
                  " Price of work: $[price]; Description: [your proposition]")

def get_bid_text(project):
    
    project_title = project['project_title'].iloc[0]
    project_description = project['project_description'].iloc[0]

    project_min_amount = project['project_min_amount'].iloc[0]
    project_max_amount = project['project_max_amount'].iloc[0]
    
    completion = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
        {"role": "system", "content": system_content},
        {"role": "user", "content": (f"Title: {project_title}\nProject desired min amount: {project_min_amount}"
                                     f"Project desired max amount: {project_max_amount}\n"
                                     f"Description: {project_description}")}
      ]
    )

    return completion.choices[0].message.content.replace('[Your Name]', '@username')

In [11]:
import random

def get_random_project():
    
    project_id = random.choice(df['project_id'].unique())
    project_bids_indexes = np.where(df['project_id'] == project_id)[0]

    project = df.iloc[project_bids_indexes]
    
    return project

In [14]:
for i in range(10):
    project = get_random_project().reset_index(drop=True)
    bid_text = get_bid_text(project)

    prob,bid = predict_generated_bid_prob(project, bid_text)
    print(f'Probability to be awarded: {prob* 100}')

    probs = predict_bids_probs(utils.prepare_df(project)) * 100
    probs_descending = np.argsort(probs)[::-1]
    
    print('Probs of others bids: ', probs[probs_descending[:5]])
    
    project_without_sealed_bids = project.drop(np.where(project['bid_sealed'])[0])
    
    probs = predict_bids_probs(utils.prepare_df(project_without_sealed_bids)) * 100
    probs_descending = np.argsort(probs)[::-1]
    
    print('Probs of not sealed bids: ', probs[probs_descending[:5]]), print()

Probability to be awarded: 21.59897786018141
Probs of others bids:  [23.5093121  23.04811598 22.37348237 22.10522152 21.95969543]
Probs of not sealed bids:  [23.5093121  23.04811598 22.37348237 22.10522152 21.95969543]

Probability to be awarded: 17.524647180729634
Probs of others bids:  [30.7250946  24.6687892  22.44288699 18.32455744 18.21524791]
Probs of not sealed bids:  [30.7250946  24.6687892  18.32455744 18.21524791 17.95715687]

Probability to be awarded: 17.913745008109032
Probs of others bids:  [25.63333811 23.73565763 18.07232737 18.02047637]
Probs of not sealed bids:  [25.63333811 23.73565763 18.07232737 18.02047637]

Probability to be awarded: 21.877167177002015
Probs of others bids:  [32.82840302 25.54024853 22.81928066 21.9317463  21.85799774]
Probs of not sealed bids:  [32.82840302 25.54024853 22.81928066 21.9317463  21.85799774]

Probability to be awarded: 17.944437587340367
Probs of others bids:  [23.6019264  18.34019352 18.18844589]
Probs of not sealed bids:  [23.601

In [15]:
bid_text

'Period of work: 10 days; Price of work: $500; Description: I propose developing a machine learning model to automate the valve control in the system. By analyzing historical data and real-time sensor readings, the model will be able to make intelligent decisions on when to switch valves to manual or automatic mode. Additionally, I will create visualizations using the built-in visualization manager to represent the system, alarms, set points, and data logging. This will provide a user-friendly interface for monitoring and controlling the system.'